In [1]:
# --- FPL bootstrap-static → pandas in a Jupyter notebook ---
import requests
import pandas as pd
from pandas import json_normalize
from time import sleep

URL = "https://fantasy.premierleague.com/api/bootstrap-static/"

def get_fpl_bootstrap(url=URL, max_retries=3, backoff=2):
    """
    Fetch FPL bootstrap-static with basic retries and a friendly User-Agent.
    Returns parsed JSON as a Python dict.
    """
    headers = {"User-Agent": "FPL-Notebook/1.0 (+https://fantasy.premierleague.com/)"}
    for attempt in range(1, max_retries + 1):
        try:
            resp = requests.get(url, headers=headers, timeout=20)
            resp.raise_for_status()
            return resp.json()
        except Exception as e:
            if attempt == max_retries:
                raise
            sleep(backoff * attempt)

data = get_fpl_bootstrap()

# Peek at the top-level keys
print("Top-level keys:", list(data.keys()))

# ---- Core tables ----
players_df        = json_normalize(data["elements"])         # player-level data
teams_df          = json_normalize(data["teams"])            # team info
events_df         = json_normalize(data["events"])           # gameweeks
element_types_df  = json_normalize(data["element_types"])    # positions (GK/DEF/MID/FWD)

# Make team id → team name map and position id → position name map
team_map = dict(zip(teams_df["id"], teams_df["name"]))
pos_map  = dict(zip(element_types_df["id"], element_types_df["singular_name_short"]))

# Add readable columns to the players table
players_df = players_df.assign(
    team_name = players_df["team"].map(team_map),
    position  = players_df["element_type"].map(pos_map),
    price     = players_df["now_cost"] / 10.0  # convert to £m
)

# Select a handy subset of player columns for quick analysis
cols = [
    "id","first_name","second_name","web_name","position","team_name",
    "price","total_points","minutes","goals_scored","assists",
    "clean_sheets","goals_conceded","ict_index","selected_by_percent","form"
]
quick_players = players_df[cols].sort_values("total_points", ascending=False).reset_index(drop=True)

# Display a few rows
quick_players.head(10)


Top-level keys: ['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'element_stats', 'element_types', 'elements']


,id,first_name,second_name,web_name,position,team_name,price,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,ict_index,selected_by_percent,form
0,7,Riccardo,Calafiori,Calafiori,DEF,Arsenal,5.6,26,134,1,2,2,0,15.4,8.3,13.0
1,8,Jurriën,Timber,J.Timber,DEF,Arsenal,5.6,24,82,2,1,1,0,20.2,6.9,12.0
2,82,Antoine,Semenyo,Semenyo,MID,Bournemouth,7.2,21,180,2,1,1,4,20.1,23.7,10.5
3,661,Hugo,Ekitiké,Ekitiké,FWD,Liverpool,8.7,20,150,2,1,0,2,19.9,26.7,10.0
4,205,Josh,Cullen,Cullen,MID,Burnley,5.0,18,180,1,1,1,3,18.7,1.6,9.0
5,565,Guglielmo,Vicario,Vicario,GKP,Spurs,5.0,18,180,0,0,2,0,5.8,7.0,9.0
6,531,Daniel,Ballard,Ballard,DEF,Sunderland,4.6,18,98,1,0,1,0,15.2,5.0,9.0
7,384,Cody,Gakpo,Gakpo,MID,Liverpool,7.6,17,180,1,2,0,4,20.4,12.8,8.5
8,249,João Pedro,Junqueira de Jesus,João Pedro,FWD,Chelsea,7.6,17,162,1,2,1,1,16.8,59.4,8.5
9,597,Richarlison,de Andrade,Richarlison,FWD,Spurs,6.7,17,148,2,1,2,0,20.5,12.8,8.5


In [11]:
import numpy as np
import pandas as pd

# --- You already have players_df with at least these columns:
# id, first_name, second_name, web_name, position, team_name, price, total_points,
# minutes, goals_scored, assists, clean_sheets, goals_conceded, ict_index,
# selected_by_percent, form
#
# We will add: DEF_SCORE_VAL, MID_SCORE_VAL, FWD_SCORE_BAL, PICK_SCORE, PICK_SCORE_0_100

# Ensure numeric for string-coded metrics (FPL ships some as strings)
to_numeric_cols = [
    "influence", "creativity", "threat", "ict_index", "form",
    "expected_goals", "expected_assists", "expected_goal_involvements"
]
for c in to_numeric_cols:
    if c in players_df.columns:
        players_df[c] = pd.to_numeric(players_df[c], errors="coerce")
    else:
        # create if missing (prevents KeyError; stays NaN if truly unavailable)
        players_df[c] = np.nan

# Minutes floor to avoid tiny-sample noise (tune as you like)
MIN_FLOOR = 180

def per90(numer, minutes):
    mins90 = np.where(minutes > 0, minutes / 90.0, np.nan)
    return numer / mins90

def zscore(series: pd.Series) -> pd.Series:
    mu = np.nanmean(series)
    sd = np.nanstd(series)
    if sd == 0 or np.isnan(sd):
        return pd.Series(np.zeros(len(series)), index=series.index)
    return (series - mu) / sd

# Initialize columns
players_df["DEF_SCORE_VAL"] = np.nan
players_df["MID_SCORE_VAL"] = np.nan
players_df["FWD_SCORE_BAL"] = np.nan

# -------------------------
# DEFENDERS — Value score
# -------------------------
defs = players_df[(players_df["position"] == "DEF") & (players_df["minutes"] >= MIN_FLOOR)].copy()

# Per-90s
defs["CS90"] = per90(defs["clean_sheets"], defs["minutes"])
defs["GC90"] = per90(defs["goals_conceded"], defs["minutes"])
defs["A90"]  = per90(defs["assists"], defs["minutes"])

# Z within defenders
z_bps   = zscore(defs.get("bps", pd.Series(np.nan, index=defs.index)))  # if 'bps' missing, becomes zeros
z_infl  = zscore(defs["influence"])
z_CS90  = zscore(defs["CS90"])
z_GC90  = zscore(defs["GC90"])
z_A90   = zscore(defs["A90"])
z_formd = zscore(defs["form"])

# Value-adjusted defender score (your chosen one)
def_val_num = (0.40*z_bps + 0.20*z_infl + 0.15*z_CS90 - 0.15*z_GC90 + 0.06*z_A90 + 0.04*z_formd)
defs["DEF_SCORE_VAL"] = def_val_num / defs["price"].replace(0, np.nan)

players_df.loc[defs.index, "DEF_SCORE_VAL"] = defs["DEF_SCORE_VAL"]

# -------------------------
# MIDFIELDERS — Value score
# -------------------------
mids = players_df[(players_df["position"] == "MID") & (players_df["minutes"] >= MIN_FLOOR)].copy()

mids["G90"]   = per90(mids["goals_scored"], mids["minutes"])
mids["A90"]   = per90(mids["assists"], mids["minutes"])
mids["xGI90"] = per90(mids["expected_goal_involvements"], mids["minutes"])

z_G90m   = zscore(mids["G90"])
z_A90m   = zscore(mids["A90"])
z_xGI90m = zscore(mids["xGI90"])
z_crem   = zscore(mids["creativity"])
z_thm    = zscore(mids["threat"])
z_formm  = zscore(mids["form"])

mid_val_num = (0.35*z_G90m + 0.25*z_A90m + 0.15*z_xGI90m + 0.10*z_crem + 0.10*z_thm + 0.05*z_formm)
mids["MID_SCORE_VAL"] = mid_val_num / mids["price"].replace(0, np.nan)

players_df.loc[mids.index, "MID_SCORE_VAL"] = mids["MID_SCORE_VAL"]

# -------------------------
# FORWARDS — Balanced score
# -------------------------
fwds = players_df[(players_df["position"] == "FWD") & (players_df["minutes"] >= MIN_FLOOR)].copy()

fwds["G90"]   = per90(fwds["goals_scored"], fwds["minutes"])
fwds["A90"]   = per90(fwds["assists"], fwds["minutes"])
fwds["xGI90"] = per90(fwds["expected_goal_involvements"], fwds["minutes"])

z_G90f   = zscore(fwds["G90"])
z_A90f   = zscore(fwds["A90"])
z_xGI90f = zscore(fwds["xGI90"])
z_thf    = zscore(fwds["threat"])
z_formf  = zscore(fwds["form"])

fwds["FWD_SCORE_BAL"] = (0.40*z_G90f + 0.20*z_A90f + 0.20*z_xGI90f + 0.10*z_thf + 0.10*z_formf)

players_df.loc[fwds.index, "FWD_SCORE_BAL"] = fwds["FWD_SCORE_BAL"]

# -------------------------------------
# Unified pick score & 0–100 display
# -------------------------------------
players_df["PICK_SCORE"] = np.nan
players_df.loc[players_df["position"] == "DEF", "PICK_SCORE"] = players_df["DEF_SCORE_VAL"]
players_df.loc[players_df["position"] == "MID", "PICK_SCORE"] = players_df["MID_SCORE_VAL"]
players_df.loc[players_df["position"] == "FWD", "PICK_SCORE"] = players_df["FWD_SCORE_BAL"]

def to_100(series):
    arr = series.astype(float).to_numpy()
    lo, hi = np.nanmin(arr), np.nanmax(arr)
    if np.isnan(lo) or np.isnan(hi) or hi - lo == 0:
        return pd.Series(np.nan, index=series.index)
    return pd.Series(100 * (series - lo) / (hi - lo), index=series.index)

players_df["PICK_SCORE_0_100"] = to_100(players_df["PICK_SCORE"])

# Optional quick checks:
players_df.loc[players_df["position"]=="DEF"].nlargest(10, "DEF_SCORE_VAL")[["web_name","team_name","price","DEF_SCORE_VAL"]]
# players_df.loc[players_df["position"]=="MID"].nlargest(10, "MID_SCORE_VAL")[["web_name","team_name","price","MID_SCORE_VAL"]]
# players_df.loc[players_df["position"]=="FWD"].nlargest(10, "FWD_SCORE_BAL")[["web_name","team_name","price","FWD_SCORE_BAL"]]
# players_df.nlargest(15, "PICK_SCORE_0_100")[["web_name","position","team_name","price","PICK_SCORE_0_100"]]


,web_name,team_name,price,DEF_SCORE_VAL
611,Romero,Spurs,5.0,0.233861
5,Saliba,Arsenal,6.1,0.232677
617,Van de Ven,Spurs,4.5,0.226378
39,Cash,Aston Villa,4.5,0.209523
505,Livramento,Newcastle,5.0,0.181500
4,Gabriel,Arsenal,6.1,0.179317
610,Pedro Porro,Spurs,5.6,0.179159
615,Spence,Spurs,4.5,0.164129
268,Lacroix,Crystal Palace,5.0,0.143926
578,Reinildo,Sunderland,4.0,0.143373


In [12]:
players_df.loc[players_df["position"]=="MID"].nlargest(10, "MID_SCORE_VAL")[["web_name","team_name","price","MID_SCORE_VAL"]]

,web_name,team_name,price,MID_SCORE_VAL
129,Semenyo,Bournemouth,7.2,0.296478
409,Gakpo,Liverpool,7.6,0.205273
95,Cullen,Burnley,5.0,0.183470
453,Reijnders,Man City,5.7,0.116855
248,Neto,Chelsea,7.0,0.076180
406,M.Salah,Liverpool,14.5,0.068464
386,Stach,Leeds,5.0,0.067677
277,Sarr,Crystal Palace,6.5,0.022816
548,Anderson,Nott'm Forest,5.5,0.019989
655,L.Paquetá,West Ham,5.9,0.011293


In [13]:
players_df.loc[players_df["position"]=="FWD"].nlargest(10, "FWD_SCORE_BAL")[["web_name","team_name","price","FWD_SCORE_BAL"]]

,web_name,team_name,price,FWD_SCORE_BAL
186,Thiago,Brentford,6.0,0.868821
667,Bowen,West Ham,7.8,-0.417716
66,Watkins,Aston Villa,9.0,-0.451105
29,Havertz,Arsenal,7.4,NaN
30,G.Jesus,Arsenal,6.5,NaN
33,Gyökeres,Arsenal,9.0,NaN
67,Redmond,Aston Villa,4.5,NaN
105,Flemming,Burnley,5.5,NaN
106,Amdouni,Burnley,4.9,NaN
107,Foster,Burnley,5.0,NaN


In [14]:
players_df.nlargest(15, "PICK_SCORE_0_100")[["web_name","position","team_name","price","PICK_SCORE_0_100"]]

,web_name,position,team_name,price,PICK_SCORE_0_100
186,Thiago,FWD,Brentford,6.0,100.000000
129,Semenyo,MID,Bournemouth,7.2,56.638229
611,Romero,DEF,Spurs,5.0,51.894260
5,Saliba,DEF,Arsenal,6.1,51.804560
617,Van de Ven,DEF,Spurs,4.5,51.327347
39,Cash,DEF,Aston Villa,4.5,50.050355
409,Gakpo,MID,Liverpool,7.6,49.728402
95,Cullen,MID,Burnley,5.0,48.076604
505,Livramento,DEF,Newcastle,5.0,47.927300
4,Gabriel,DEF,Arsenal,6.1,47.761900


In [15]:
# Export the full players_df to CSV for Tableau
players_df.to_csv("fpl_players_with_scores.csv", index=False)

print("✅ Data exported to fpl_players_with_scores.csv")

✅ Data exported to fpl_players_with_scores.csv
